In [18]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve

In [19]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [20]:
simp = False;

## Target function

we want to minimize the effective price, given a set of Deltas for the buying
and selling asset each.

In [21]:
def spotByDelta_(asset, s, v, b, w, j, e, Delta, a, min):
  if asset == 'b':
    d = -Delta
  else:
    d = Delta
  f = (v + b + d) * w
  if simp:
    f = simplify(f)
  return f

spotByDelta = {asset: spotByDelta_(asset, *all_symbols[asset]) for asset in assets}
spotByDelta['b']

w_b*(Delta_b + b_b + v_b)

In [ ]:
spotByDelta["s"];

In [22]:
def expBySpot_(s, v, b, w, j, e, Delta, a, min):
  f = log(s/a) / log(1 + 1/j)
  if simp:
    f = simplify(f)
  return f
expBySpot = {asset: expBySpot_(*all_symbols[asset]) for asset in assets}
expBySpot['b']

log(s_b/a_b)/log(1 + 1/j_b)

In [23]:
def expByDelta_(asset, s, v, b, w, j, e, Delta, a, min):
    f = expBySpot[asset].subs(s, spotByDelta[asset])
    if simp:
        f = simplify(f)
    return f

expByDelta = {asset: expByDelta_(asset, *all_symbols[asset]) for asset in assets}
expByDelta['b']

log(w_b*(Delta_b + b_b + v_b)/a_b)/log(1 + 1/j_b)

In [24]:
eff = Delta_s / Delta_b;
eff;

-Delta_s/Delta_b

### convexity of the target function

- deltaBySpot is linear
- spotByExp is convex
- deltaByExp is a composition of the two

=> deltaByExp is convex

- negation flips convexity/concavity
- deltaBuyingByExp is negative => -deltaBuyingByExp is positive
- inversion of a positive function flips convexity/concavity => 1 /
  (-deltaBuyingByExp) is convex
- multiplying two nonnegative functions with different variables, that are each
  convex and independent of the other's variable, results in a convex function

==> effectivePrice = deltaSellingByExp / (-deltaBuyingByExp) is convex. qed

## inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds
  given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not
  exceed expLimit (TODO)

In [25]:
a0Buying = spotByDelta['s'] * -Delta_b
if simp:
    a0Buying = simplify(a0Buying)
a0Buying

-Delta_b*w_s*(Delta_s + b_s + v_s)

In [26]:
a0Selling = spotByDelta['b'] * Delta_s
if simp:
    a0Selling = simplify(a0Selling)
a0Selling

Delta_s*w_b*(Delta_b + b_b + v_b)

In [27]:
a0Bound = a0Buying - a0Selling # <=! 0
if simp:
    a0Bound = simplify(a0Bound)
a0Bound

-Delta_b*w_s*(Delta_s + b_s + v_s) - Delta_s*w_b*(Delta_b + b_b + v_b)

In [28]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    exp = expByDelta[asset]
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = exp - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - exp # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

-log(w_b*(b_b + v_b)/a_b)/log(1 + 1/j_b) + log(w_b*(Delta_b + b_b + v_b)/a_b)/log(1 + 1/j_b)

In [29]:
expBound["s"];

log(w_s*(b_s + v_s)/a_s)/log(1 + 1/j_s) - log(w_s*(Delta_s + b_s + v_s)/a_s)/log(1 + 1/j_s)

In [30]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

def maxSpotBound_(asset, s, v, b, w, j, e, Delta, a, min):
  f = spotByDelta[asset] - I_max # <=! 0
  if simp:
    f = simplify(f)
  return f

maxSpotBound = {asset: maxSpotBound_(asset, *all_symbols[asset]) for asset in assets}
maxSpotBound['b']

-I_max + w_b*(Delta_b + b_b + v_b)

In [31]:
def minAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    if asset == 'b':
        f = -min - Delta # <=! 0
    else:
        f = min - Delta # <=! 0
    if simp:
        f = simplify(f)
    return f

minAmntBound = {asset: minAmntBound_(asset, *all_symbols[asset]) for asset in assets}
minAmntBound['b']

-Delta_b - min_b

In [32]:
def maxAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    if asset == 'b':
        f = -Delta - b # <=! 0
    else:
        f = Delta - max_s # <=! 0
    if simp:
        f = simplify(f)
    return f

maxAmntBound = {asset: maxAmntBound_(asset, *all_symbols[asset]) for asset in assets}
maxAmntBound['b']

-Delta_b - b_b

In [33]:
maxAmntBound["s"];

Delta_s - max_s

## Lagrangian

In [34]:
# Decision variables
x1, x2 = Delta_b, Delta_s

# Constraints
g = [
    expBound['b'],
    expBound['s'],
    maxSpotBound['b'],
    minAmntBound['b'],
    maxAmntBound['b'],
    minAmntBound['s'],
    maxAmntBound['s'],
    a0Bound,
] 

# Lagrange multipliers for m constraints
lambdas = symbols('lambda1:{}'.format(len(g)+1), nonnegative=True)

# Objective function
f = effByExps

# Lagrangian
L = f + sum([lambdas[i] * g[i] for i in range(len(g))])
if simp:
    L = simplify(L)

# Differentiate Lagrangian with respect to decision variables
dL_dx1 = L.diff(x1)
dL_dx2 = L.diff(x2)
if simp:
    dL_dx1 = simplify(dL_dx1)
    dL_dx2 = simplify(dL_dx2)

# Equations from stationarity
equations = [Eq(dL_dx1, 0), Eq(dL_dx2, 0)]

# Complementary slackness conditions
for i in range(len(g)):
    equations.append(Eq(lambdas[i] * g[i], 0))

# Solve
solutions = nonlinsolve(equations, [x1, x2] + list(lambdas))
solutions

NameError: name 'effByExps' is not defined